<a href="https://colab.research.google.com/github/testgithubprecious/Ml_projects/blob/main/3d_voxel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Install required libraries if not already:
# pip install torch torchvision matplotlib numpy trimesh scikit-image scipy

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import trimesh
from scipy.ndimage import zoom
import matplotlib.pyplot as plt

# ------------------------------
# Function: Convert mesh to voxel grid
# ------------------------------
def mesh_to_voxel(file_path, grid_size=32):
    mesh = trimesh.load(file_path)
    voxel = mesh.voxelized(pitch=mesh.scale / grid_size).matrix.astype(np.float32)
    # Normalize to fixed size
    voxel = zoom(voxel, (grid_size / voxel.shape[0],) * 3)
    return voxel

# ------------------------------
# Example usage: load a sample .off mesh
# Replace with your own file path
# ------------------------------
file = 'data/ModelNet10/chair/train/chair_0001.off'
vox = mesh_to_voxel(file)  # Shape: (32, 32, 32)
vox_tensor = torch.tensor(vox).unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, 32, 32, 32)

# ------------------------------
# 3D CNN Model for volumetric classification
# ------------------------------
class VoxelCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(VoxelCNN, self).__init__()
        self.conv1 = nn.Conv3d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool3d(2)
        self.fc1 = nn.Linear(64 * 8 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # -> (B, 32, 16, 16, 16)
        x = self.pool(F.relu(self.conv2(x)))  # -> (B, 64, 8, 8, 8)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# ------------------------------
# Initialize model and test
# ------------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VoxelCNN(num_classes=10).to(device)

# Forward pass
output = model(vox_tensor.to(device))
print("🔍 Predicted class logits:", output.detach().cpu().numpy())